In [7]:
%%time
import time
time.sleep(2)
# Provides ways to work with large multidimensional arrays
import numpy as np 

# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline


# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

# import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

from IPython.display import display
# Setting display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

import os
import sys
import seaborn as sns
# from pathlib import Path
from quant_free.dataset.us_equity_load import *
from quant_free.utils.us_equity_utils import *

from quant_free.factor.xq_finance import *
from quant_free.factor.index import *
from quant_free.dataset.us_equity_load import *
from quant_free.factor.price import*
# %load_ext ipython_magic
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2
# %load_ext ipython_magic
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.instance().extension_manager.load_extension("autoreload")
# InteractiveShell.instance().extension_manager.load_extension("writefile")
import sys
print(sys.executable)


# import IPython
# print(IPython.__version__)
# from IPython.core.magic import register_line_cell_magic

# @register_line_cell_magic
# def writefile(line, cell=None):
#     with open(line, 'w') as f:
#         f.write(cell or '')


folder_name = './factor'
file_name = 'index.py'
# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
file_path = os.path.join('./', 'factor', 'index.py')


start_date = '2020-01-01'
end_date = '2024-09-28'

/opt/openbb/bin/python
CPU times: user 1.85 ms, sys: 7.97 ms, total: 9.81 ms
Wall time: 2.01 s


In [10]:
# %%writefile $file_path

from dataset.us_equity_load import *

class index:
  def __init__(self, sectors, start_date, end_date, trade_option = 'market_capital', dir = 'fh'):
     self.sectors = sectors
     self.start_date = start_date
     self.end_date = end_date
     self.dir = dir
     self.trade_option = trade_option

  def calc(self):
      dict_index = {}
      for sector in self.sectors:
        print(f"processing {sector} ...")
        data_symbols = us_dir1_load_csv(dir0 = 'symbol', dir1 = self.dir, filename= sector +'.csv')
        if (data_symbols.empty == False):
          symbols = data_symbols['symbol'].values

          data = us_equity_daily_data_load(symbols = symbols, start_date = self.start_date,
                                            end_date = self.end_date, trade_option = self.trade_option, 
                                            dir_option = 'xq')
          if (len(data) > 0):
            df = pd.DataFrame(data)
            df_sum = df.sum(axis=1)
            index = df_sum * 1000 /df_sum.iloc[0]
            dict_index[sector] = index
          else:
            self.sectors.remove(sector)
        else:
           print(f"remove {sector} ...")
           self.sectors.remove(sector)

      df = pd.DataFrame(dict_index)
      return df

  def ratio(self, days = 1):
     df = self.calc()
     df = df.fillna(0)
     return df.pct_change(days).round(5)

# xueqiu sector

In [ ]:
sector_file = 'us_equity_sector.csv'
sectors = list(us_dir1_load_csv(dir0 = 'symbol', dir1 = 'xq', filename=sector_file)['name'].values)


print(sectors)

df_index = index(sectors, start_date, end_date, trade_option = 'market_capital', dir = 'xq')
df = df_index.ratio()
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'xq', filename='xq_index_price_ratio' + '.csv', data = df)

df_index = index(sectors, start_date, end_date, trade_option = 'amount', dir = 'xq')
df = df_index.ratio()
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'xq', filename='xq_index_amount_ratio' + '.csv', data = df)

# fh sector

In [ ]:

sector_file = 'us_equity_sector.csv'
sectors = list(us_dir1_load_csv(dir0 = 'symbol', dir1 = 'fh', filename=sector_file)['Sector'].values)

print(sectors)

df_index = index(sectors, start_date, end_date, trade_option = 'market_capital', dir = 'fh')
df = df_index.ratio()
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'fh', filename='fh_index_price_ratio' + '.csv', data = df)

df_index = index(sectors, start_date, end_date, trade_option = 'amount', dir = 'fh')
df = df_index.ratio()
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'fh', filename='fh_index_amount_ratio' + '.csv', data = df)

In [ ]:
print(df)